In [ ]:
import os
import random
import torch
import mne

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.utils import shuffle
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch.utils.data import random_split, TensorDataset
from torch_geometric.utils import to_undirected
from scipy.spatial.distance import cdist
from scipy.stats import skew, kurtosis
from tqdm import tqdm
from utils.models import *
from utils.functions import *


#### ready for the dataset
* channels and sequences

    'Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3','AF4','AF8', 'F7', 'F5','F3','F1','Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1','FCz','FC2','FC4', 'FC6', 'FT8', 'T7','C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1','CPz','CP2', 'CP4','CP6', 'TP8', 'P7','P5', 'P3', 'P1', 'Pz','P2', 'P4', 'P6', 'P8', 'PO7', 'PO3','POz', 'PO4','PO8', 'O1','Oz','O2', 'F9', 'F10', 'TP9', 'TP10'
* After reordering, the sequence for both imagery and video trials is as follows:

    reorder = ['sad4', 'sad5', 'sad8', 'dis4', 'dis5', 'dis8', 'fear4', 'fear5', 'fear8', 'neu4', 'neu5', 'neu8', 'joy4', 'joy5', 'joy8', 'ten4', 'ten5', 'ten8', 'ins4', 'ins5', 'ins8']
* emotion labels: 
    * negative(sadness, disgust, fear):             0
    * positive(happiness, inspiration, tenderness)：1
    * neutral:                                      2

* so the reoder labels sequence should be: [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]


#### load coords

In [ ]:
##--------------------------load mne------------------------------##
# load biosemi64 electrode layout
montage = mne.channels.make_standard_montage('standard_1005')
positions = montage.get_positions()

## electrode name and 3-dimensional coordinates
ch_pos = positions['ch_pos']

df = pd.DataFrame.from_dict(ch_pos, orient = 'index', columns = ['x', 'y','z'])
df = df.reset_index().rename(columns = {'index': 'electrode'})
# print (df)

##------------------------load channels --------------------------##
channels = ['Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3', 'AF4', 'AF8', 'F7',
            'F5',  'F3',  'F1',  'Fz',  'F2',  'F4',  'F6',  'F8', 
            'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 
            'FT8', 'T7',  'C5',  'C3',  'C1',  'Cz',  'C2',  'C4', 
            'C6',  'T8',  'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2',
            'CP4', 'CP6', 'TP8', 'P7',  'P5',  'P3',  'P1',  'Pz',
            'P2',  'P4',  'P6',  'P8',  'PO7', 'PO3', 'POz', 'PO4',
            'PO8', 'O1',  'Oz',  'O2',  'F9',  'F10', 'TP9', 'TP10']

##-----------------calculate Euclidean distance----------------------##
# p1 = np.array([x1, y1, z1])
# distance = cdist(mat1, mat2)
used_channel = channels[: 64]

# build a map 
coord_map = {ch: ch_pos[ch] for ch in used_channel}
coords = np.array([coord_map[ch] for ch in used_channel])

# now calculate the distance
dist_mat = cdist(coords, coords)

# dist_df = pd.DataFrame(dist_mat, index=used_channel, columns=used_channel)
dist_tensor = torch.tensor(dist_mat, dtype=torch.float)


### CNN

#### 1. prepare datum

In [ ]:
basic_pth = r'EEG_data\reordered'
all_data = []
all_labels = []
len_data = len(os.listdir(basic_pth))

def get_labels_for_subject(num0, num1, num2):
    return np.array([0]*num0 + [2]*num2 + [1]*num1)

for i in range(1, len_data+1):
    data = np.load(f'{basic_pth}\sub-{i:02d}_ses-ima_task-emotion_reorder.npy')
    data = np.transpose(data, (1, 0, 2))
    label = get_labels_for_subject(9, 3, 9) 
    
    all_data.append(data)
    all_labels.append(label)


## concat data 
data = torch.tensor(np.concatenate(all_data, axis=0), dtype=torch.float)      # shape: [21*40=840, 64, 6000]
labels = torch.tensor(np.concatenate(all_labels, axis=0),dtype=torch.long )   # shape: [21*40=840]
   
input_lst = TensorDataset(data, labels)

# split Data into train_set and test_set
train_len = int(0.8 * len(input_lst))
test_len = len(input_lst) - train_len
train_dataset, test_dataset = random_split(input_lst, [train_len, test_len])

train_loader = DataLoader(train_dataset, batch_size=21, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=21)

#### 2. run this model

In [34]:
model = SimpleCCN(in_channels = 64, hidden_channels = 128, 
                  kernel_size=5, out_channel = 3, n_drop = 0.7)

res_acc = fit(model, 'CNN', train_loader = train_loader, 
              test_loader = test_loader, epochs = 10, lr =  1e-3)


Testing: 100%|██████████| 8/8 [00:00<00:00,  9.63it/s]


Epoch 000, Train Loss: 3.6746, Test Acc: 0.4345


Testing: 100%|██████████| 8/8 [00:00<00:00, 11.26it/s]


Epoch 001, Train Loss: 1.0680, Test Acc: 0.4345


Testing: 100%|██████████| 8/8 [00:00<00:00, 12.02it/s]


...

Epoch 007, Train Loss: 1.0161, Test Acc: 0.4345


Testing: 100%|██████████| 8/8 [00:00<00:00, 11.46it/s]


Epoch 008, Train Loss: 1.0091, Test Acc: 0.4345


Testing: 100%|██████████| 8/8 [00:00<00:00, 11.88it/s]

Epoch 009, Train Loss: 1.0122, Test Acc: 0.4345


### GCN

#### 1. prepare datum

In [56]:
##-------------------build feature matrix-----------------------------##
basic_pth = r'EEG_data\reordered'
all_data = []
len_data = len(os.listdir(basic_pth))

for i in range(1, len_data+1):
    data = np.load(f'{basic_pth}\sub-{i:02d}_ses-ima_task-emotion_reorder.npy')
    data = np.transpose(data, (1, 0, 2))
    
    all_data.append(data)

## concat data 
data = np.concatenate(all_data, axis=0)  # [840, 64, 6000]
num_trials, num_channels, num_samples = data.shape

features = np.zeros((num_trials, num_channels, 4))
features[:, :, 0] = np.mean(data, axis=2)
features[:, :, 1] = np.std(data, axis=2)
features[:, :, 2] = skew(data, axis=2)
features[:, :, 3] = kurtosis(data, axis=2)
features_tensor = torch.tensor(features, dtype=torch.float) 

## adjacency mat
adj_mat = torch.tensor(use_method(dist_mat, 'knn', k=5), dtype=torch.float )  

## feature mat 
feature_mat = features_tensor 

## true emotion labels
y = torch.tensor(([0]*9 + [2]*3 + [1]*9) * 40).long() 

# connection edge
edge_index_np = np.array(np.nonzero(adj_mat))
edge_index = torch.tensor(edge_index_np, dtype=torch.long)
edge_index = to_undirected(edge_index)

In [57]:
# split Data into train_set and test_set
input_lst = []
for i in range(len(y)):
    x = feature_mat[i]
    y_i = y[i]
    data_i = Data(x=x, edge_index=edge_index.clone(), y=y_i)  ## remember to clone!
    input_lst.append(data_i)

train_len = int(0.8 * len(input_lst))
test_len = len(input_lst) - train_len
train_dataset, test_dataset = random_split(input_lst, [train_len, test_len])

train_loader = DataLoader(train_dataset, batch_size = 21, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 21)

####  2. now run this model

In [ ]:
model = SimpleGCN(in_channels = 4, hidden_channels = 256,
                  fc_hidden = 512, num_classes = 3, n_drop = 0.7)
res_acc = fit(model, 'GCN', train_loader = train_loader, 
              test_loader = test_loader, epochs = 10, lr = 1e-4)


Testing: 100%|██████████| 8/8 [00:00<00:00, 111.92it/s]


Epoch 000, Train Loss: 1.0591, Test Acc: 0.3929


Testing: 100%|██████████| 8/8 [00:00<00:00, 125.19it/s]


Epoch 001, Train Loss: 1.0410, Test Acc: 0.3750


Testing: 100%|██████████| 8/8 [00:00<00:00, 125.93it/s]


...

Epoch 007, Train Loss: 1.0230, Test Acc: 0.3988


Testing: 100%|██████████| 8/8 [00:00<00:00, 116.67it/s]


Epoch 008, Train Loss: 1.0051, Test Acc: 0.3750


Testing: 100%|██████████| 8/8 [00:00<00:00, 129.65it/s]

Epoch 009, Train Loss: 1.0290, Test Acc: 0.4107


### CNN(output feature) + GCN(input feature)

#### 1. prepare datum (from CNN)

In [48]:
basic_pth = r'EEG_data\reordered'
all_data = []
all_labels = []
len_data = len(os.listdir(basic_pth))

def get_labels_for_subject(num0, num1, num2):
    return np.array([0]*num0 + [2]*num2 + [1]*num1)

for i in range(1, len_data+1):
    data = np.load(f'{basic_pth}\sub-{i:02d}_ses-ima_task-emotion_reorder.npy')
    data = np.transpose(data, (1, 0, 2))
    label = get_labels_for_subject(9, 3, 9) 
    
    all_data.append(data)
    all_labels.append(label)

## concat data 
data = torch.tensor(np.concatenate(all_data, axis=0), dtype=torch.float) 
labels = torch.tensor(np.concatenate(all_labels, axis=0),dtype=torch.long )   # shape: [21*40=840]

batch_size, n_channels, seq_len = data.shape

# 2. reshape 
data_reshaped = data.reshape(batch_size * n_channels, 1, seq_len)

# 3. expand label
y_expanded = labels.unsqueeze(1).repeat(1, n_channels).reshape(-1)

# 4. build Dataset and Loader
input_lst = TensorDataset(data_reshaped, y_expanded)
loader = DataLoader(input_lst, batch_size=21, shuffle=False)

# 5. define model
model = SimpleCCN(in_channels=1, hidden_channels=64, kernel_size=5, out_channel=4, n_drop=0.5)

# 6. generate feature
def generate_fea(model, loader):
    model.eval()
    preds = []
    with torch.no_grad():
        for inputs, _ in loader:
            out = model(inputs)
            prediction = nn.functional.softmax(out, dim=1)
            preds.append(prediction)
    return torch.cat(preds, dim=0)

features = generate_fea(model, loader)  # (21*64*40, 4)
CNNfeature_mat = features.reshape(batch_size, n_channels, -1)  # (21, 64, 4)


#### 2. into GNN format

In [49]:
## adjacency mat
adj_mat = torch.tensor(use_method(dist_mat, 'knn', k=5), dtype=torch.float )  

## feature mat 
feature_mat = CNNfeature_mat   # feature， Should be the exact value([mean, std, skewness, kurtosis])

## true emotion labels
y = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]) 

# connection edge
edge_index_np = np.array(np.nonzero(adj_mat))
edge_index = torch.tensor(edge_index_np, dtype=torch.long)
edge_index = to_undirected(edge_index)

In [50]:
# split Data into train_set and test_set
input_lst = []
for i in range(len(y)):
    x = feature_mat[i]
    y_i = y[i]
    data_i = Data(x=x, edge_index=edge_index.clone(), y=y_i)  ## remember to clone!
    input_lst.append(data_i)

train_len = int(0.8 * len(input_lst))
test_len = len(input_lst) - train_len
train_dataset, test_dataset = random_split(input_lst, [train_len, test_len])

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1)

#### 3. run this model

In [55]:

model = SimpleGCN(in_channels = 4, hidden_channels = 256, 
                  fc_hidden = 512, num_classes = 3, n_drop = 0.7)

res_acc = fit(model, 'GCN', train_loader = train_loader, 
              test_loader = test_loader, epochs = 10, lr = 1e-4)


Training:   0%|          | 0/16 [00:00<?, ?it/s]

Testing: 100%|██████████| 5/5 [00:00<00:00, 172.68it/s]


Epoch 000, Train Loss: 1.0940, Test Acc: 0.6000


Testing: 100%|██████████| 5/5 [00:00<00:00, 214.09it/s]


Epoch 001, Train Loss: 1.0901, Test Acc: 0.6000


...

Epoch 007, Train Loss: 1.0786, Test Acc: 0.6000


Testing: 100%|██████████| 5/5 [00:00<00:00, 185.95it/s]


Epoch 008, Train Loss: 1.0810, Test Acc: 0.6000


Testing: 100%|██████████| 5/5 [00:00<00:00, 211.31it/s]

Epoch 009, Train Loss: 1.0778, Test Acc: 0.6000
